## Usa colab

In [ ]:
# Installazione librerie principali
!pip install -q ultralytics fiftyone opencv-python-headless
!pip install -q matplotlib seaborn plotly scikit-learn pandas
!pip install -q transformers datasets # Per il bonus HuggingFace

print("✅ Installazione completata!")

In [ ]:
# Import librerie
import os
import json
import shutil
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm.auto import tqdm

# Deep Learning
import torch
import torchvision
from ultralytics import YOLO
import ultralytics

# FiftyOne per dataset
import fiftyone as fo
import fiftyone.zoo as foz

# Sklearn per metriche aggiuntive
from sklearn.metrics import confusion_matrix, classification_report

# Configurazione visualizzazioni
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Verifica GPU
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"\n📊 Ultralytics version: {ultralytics.__version__}")

In [ ]:
# Configurazione globale del progetto
CONFIG = {
    'classes': ['Horse', 'Lizard', 'Mouse', 'Hamster', 'Mule'],
    'num_classes': 5,
    'dataset_name': 'animals-yolo',
    'base_path': Path('/content/datasets'),
    'min_images_per_class': 10,
    'min_test_images': 100,
    'train_ratio': 0.70,
    'val_ratio': 0.15,
    'test_ratio': 0.15,
    'seed': 42
}

# Creazione struttura directory
CONFIG['base_path'].mkdir(parents=True, exist_ok=True)
print("✅ Configurazione inizializzata")
print(f"📁 Base path: {CONFIG['base_path']}")
print(f"🎯 Classi: {', '.join(CONFIG['classes'])}")

In [ ]:
# Download dataset da OpenImages v7
# Scaricheremo solo le immagini che contengono almeno una delle nostre classi
# OpenImages supporta sia detection che segmentation masks

print("🚀 Inizio download da OpenImages...")
print(f"📦 Classi richieste: {CONFIG['classes']}\n")

# Download del dataset con le classi specificate
# max_samples=None per scaricare tutte le immagini disponibili (full version)
# Questo può richiedere diversi minuti
dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",  # Useremo il train split di OpenImages, poi faremo il nostro split
    label_types=["detections", "segmentations"],  # Scarica sia bbox che maschere
    classes=CONFIG['classes'],
    max_samples=None,  # Full version: scarica tutto
    dataset_name="openimages_animals"
)

print(f"\n✅ Dataset scaricato!")
print(f"📊 Numero totale immagini: {len(dataset)}")
print(f"📁 Dataset name: {dataset.name}")

In [ ]:
# Analisi esplorativa del dataset
print("📊 ANALISI ESPLORATIVA DEL DATASET\n")
print("="*60)

# Conteggio immagini per classe
class_counts = {}
for sample in dataset:
    if sample.detections is not None:
        for detection in sample.detections.detections:
            label = detection.label
            class_counts[label] = class_counts.get(label, 0) + 1

print("\n🎯 Numero di annotazioni (bounding boxes) per classe:")
for cls in CONFIG['classes']:
    count = class_counts.get(cls, 0)
    print(f"   {cls:12s}: {count:4d} annotazioni")

print(f"\n📸 Totale immagini nel dataset: {len(dataset)}")

# Verifica disponibilità segmentation masks
samples_with_segm = sum(1 for s in dataset if s.segmentations is not None and len(s.segmentations.detections) > 0)
print(f"🎨 Immagini con segmentation masks: {samples_with_segm}")

print("="*60)

In [ ]:
# Visualizzazione di alcune immagini di esempio
# Questo ci aiuta a capire la qualità e varietà del dataset

import matplotlib.patches as mpatches

def visualize_sample(sample, title=""):
    """Visualizza un sample con bounding boxes"""
    img = cv2.imread(sample.filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    # Disegna bounding boxes
    if sample.detections:
        colors = plt.cm.tab10(np.linspace(0, 1, len(CONFIG['classes'])))
        class_colors = {cls: colors[i] for i, cls in enumerate(CONFIG['classes'])}
        
        for det in sample.detections.detections:
            # Coordinate normalizzate -> pixel
            x, y, w, h = det.bounding_box
            x, y, w, h = x * img.shape[1], y * img.shape[0], w * img.shape[1], h * img.shape[0]
            
            color = class_colors.get(det.label, (1, 1, 1))
            rect = mpatches.Rectangle(
                (x, y), w, h,
                linewidth=3,
                edgecolor=color,
                facecolor='none'
            )
            ax.add_patch(rect)
            
            # Label
            ax.text(
                x, y - 10,
                det.label,
                bbox=dict(boxstyle='round', facecolor=color, alpha=0.7),
                fontsize=10,
                color='white',
                fontweight='bold'
            )
    
    plt.tight_layout()
    plt.show()

# Mostra 3 esempi casuali per ogni classe
print("🖼️  VISUALIZZAZIONE ESEMPI DEL DATASET\n")

for target_class in CONFIG['classes']:
    # Trova un sample che contiene la classe target
    for sample in dataset:
        if sample.detections:
            labels = [d.label for d in sample.detections.detections]
            if target_class in labels:
                visualize_sample(sample, f"Esempio: {target_class}")
                break

In [ ]:
# Split del dataset in Train/Val/Test
import random

print("✂️  SPLITTING DEL DATASET\n")
print("="*60)

# Imposta seed per riproducibilità
random.seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])

# Calcola le dimensioni degli split
total_samples = len(dataset)
train_size = int(total_samples * CONFIG['train_ratio'])
val_size = int(total_samples * CONFIG['val_ratio'])
test_size = total_samples - train_size - val_size

print(f"📊 Totale immagini: {total_samples}")
print(f"   Train: {train_size} ({CONFIG['train_ratio']*100:.0f}%)")
print(f"   Val:   {val_size} ({CONFIG['val_ratio']*100:.0f}%)")
print(f"   Test:  {test_size} ({CONFIG['test_ratio']*100:.0f}%)")

# Crea gli split usando FiftyOne
# Questo assicura che le immagini non si sovrappongano tra i set
import fiftyone.utils.random as four

four.random_split(
    dataset,
    {
        "train": CONFIG['train_ratio'],
        "val": CONFIG['val_ratio'],
        "test": CONFIG['test_ratio']
    },
    seed=CONFIG['seed']
)

# Verifica gli split
train_view = dataset.match_tags("train")
val_view = dataset.match_tags("val")
test_view = dataset.match_tags("test")

print(f"\n✅ Split completato!")
print(f"   Train samples: {len(train_view)}")
print(f"   Val samples:   {len(val_view)}")
print(f"   Test samples:  {len(test_view)}")

# Verifica vincoli sul test set
print(f"\n🔍 VERIFICA VINCOLI TEST SET:")
print(f"   Minimo richiesto: {CONFIG['min_test_images']} immagini")
print(f"   Ottenuto: {len(test_view)} immagini")

if len(test_view) >= CONFIG['min_test_images']:
    print("   ✅ Vincolo soddisfatto!")
else:
    print(f"   ⚠️  ATTENZIONE: Test set troppo piccolo!")

# Verifica distribuzione classi nel test set
test_class_counts = {}
for sample in test_view:
    if sample.detections:
        for det in sample.detections.detections:
            test_class_counts[det.label] = test_class_counts.get(det.label, 0) + 1

print(f"\n📊 Distribuzione classi nel TEST SET:")
all_satisfied = True
for cls in CONFIG['classes']:
    count = test_class_counts.get(cls, 0)
    status = "✅" if count >= CONFIG['min_images_per_class'] else "⚠️"
    print(f"   {status} {cls:12s}: {count:3d} annotazioni")
    if count < CONFIG['min_images_per_class']:
        all_satisfied = False

print("="*60)

In [ ]:
# Export del dataset in formato YOLO
print("💾 EXPORT IN FORMATO YOLO\n")
print("="*60)

# Path di export
export_dir = CONFIG['base_path'] / CONFIG['dataset_name']
export_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Directory di export: {export_dir}\n")

# Check if segmentation data is available
has_segmentation = False
for sample in train_view.head(10):
    if sample.segmentations and len(sample.segmentations.detections) > 0:
        has_segmentation = True
        break

if has_segmentation:
    print("✅ Dataset ha segmentation masks - Export completo\n")
    label_field = "segmentations"
else:
    print("⚠️  Dataset NON ha segmentation masks - Export solo detection\n")
    print("   Nota: Training segmentation non sarà possibile con questo dataset")
    print("   Soluzione: Scarica nuovamente con segmentation o usa solo detection\n")
    label_field = "detections"

# Export per ogni split
for split_name, split_view in [("train", train_view), ("val", val_view), ("test", test_view)]:
    print(f"📦 Esportando {split_name}...")
    
    # FiftyOne può esportare direttamente in formato YOLO
    split_view.export(
        export_dir=str(export_dir),
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,  # usa segmentations se disponibili
        split=split_name,
        classes=CONFIG['classes']
    )
    
    print(f"   ✅ {split_name} esportato: {len(split_view)} immagini")

print(f"\n✅ Export completato!")
print(f"📊 Label field usato: {label_field}")
print("="*60)

In [ ]:
# Creazione del file data.yaml per YOLO
# Questo file è essenziale per il training di YOLO

yaml_path = export_dir / "data.yaml"

yaml_content = f"""# Dataset configuration for YOLO
# Auto-generated for Computer Vision Project

path: {export_dir}  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val      # val images (relative to 'path')
test: images/test    # test images (relative to 'path')

# Classes
names:
  0: Horse
  1: Lizard
  2: Mouse
  3: Hamster
  4: Mule

nc: 5  # number of classes
"""

with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("📄 File data.yaml creato:\n")
print(yaml_content)
print(f"💾 Salvato in: {yaml_path}")

# Verifica struttura finale
print("\n📁 STRUTTURA DATASET FINALE:")
for root, dirs, files in os.walk(export_dir):
    level = root.replace(str(export_dir), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:3]:  # Mostra solo i primi 3 files per directory
        print(f'{subindent}{file}')
    if len(files) > 3:
        print(f'{subindent}... and {len(files)-3} more files')

In [ ]:
# Import librerie
import os
import json
import shutil
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm.auto import tqdm

import torch
import torchvision
from ultralytics import YOLO
import ultralytics

import fiftyone as fo
import fiftyone.zoo as foz

from sklearn.metrics import confusion_matrix, classification_report

plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print(f"Versione PyTorch: {torch.__version__}")
print(f"CUDA disponibile: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Versione Ultralytics: {ultralytics.__version__}")

# Configurazione globale del progetto
CONFIG = {
    'classes': ['Horse', 'Lizard', 'Mouse', 'Hamster', 'Mule'],
    'num_classes': 5,
    'dataset_name': 'animals-yolo',
    'base_path': Path('/content/datasets'),
    'min_images_per_class': 10,
    'min_test_images': 100,
    'train_ratio': 0.70,
    'val_ratio': 0.15,
    'test_ratio': 0.15,
    'seed': 42
}

CONFIG['base_path'].mkdir(parents=True, exist_ok=True)
print("Configurazione inizializzata")
print(f"Percorso base: {CONFIG['base_path']}")
print(f"Classi: {', '.join(CONFIG['classes'])}")

# Download dataset da OpenImages v7
print("\nInizio download da OpenImages...")
print(f"Classi richieste: {CONFIG['classes']}\n")

dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections", "segmentations"],
    classes=CONFIG['classes'],
    max_samples=None,
    dataset_name="openimages_animals"
)

print(f"Dataset scaricato!")
print(f"Numero totale immagini: {len(dataset)}")
print(f"Nome dataset: {dataset.name}")

# Analisi esplorativa del dataset
print("\nANALISI ESPLORATIVA DEL DATASET\n")
print("="*60)

class_counts = {}
for sample in dataset:
    if sample.detections is not None:
        for detection in sample.detections.detections:
            label = detection.label
            class_counts[label] = class_counts.get(label, 0) + 1

print("\nAnnotazioni per classe:")
for cls in CONFIG['classes']:
    count = class_counts.get(cls, 0)
    print(f"   {cls:12s}: {count:4d} annotazioni")

print(f"\nTotale immagini: {len(dataset)}")

samples_with_segm = sum(1 for s in dataset if s.segmentations is not None and len(s.segmentations.detections) > 0)
print(f"Immagini con maschere segmentazione: {samples_with_segm}")

print("="*60)

# Funzione visualizzazione
import matplotlib.patches as mpatches

def visualizza_sample(sample, titolo=""):
    """Visualizza un campione con bounding boxes"""
    img = cv2.imread(sample.filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(titolo, fontsize=14, fontweight='bold')
    
    if sample.detections:
        colors = plt.cm.tab10(np.linspace(0, 1, len(CONFIG['classes'])))
        class_colors = {cls: colors[i] for i, cls in enumerate(CONFIG['classes'])}
        
        for det in sample.detections.detections:
            x, y, w, h = det.bounding_box
            x, y, w, h = x * img.shape[1], y * img.shape[0], w * img.shape[1], h * img.shape[0]
            
            color = class_colors.get(det.label, (1, 1, 1))
            rect = mpatches.Rectangle((x, y), w, h, linewidth=3, edgecolor=color, facecolor='none')
            ax.add_patch(rect)
            
            ax.text(x, y - 10, det.label, bbox=dict(boxstyle='round', facecolor=color, alpha=0.7),
                   fontsize=10, color='white', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Mostra esempi
print("ESEMPI DI VISUALIZZAZIONE\n")

for target_class in CONFIG['classes']:
    for sample in dataset:
        if sample.detections:
            labels = [d.label for d in sample.detections.detections]
            if target_class in labels:
                visualizza_sample(sample, f"Esempio: {target_class}")
                break

# Split del dataset
import random

print("\nDIVISIONE DEL DATASET\n")
print("="*60)

random.seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])

total_samples = len(dataset)
train_size = int(total_samples * CONFIG['train_ratio'])
val_size = int(total_samples * CONFIG['val_ratio'])
test_size = total_samples - train_size - val_size

print(f"Totale immagini: {total_samples}")
print(f"   Addestramento: {train_size} ({CONFIG['train_ratio']*100:.0f}%)")
print(f"   Validazione:   {val_size} ({CONFIG['val_ratio']*100:.0f}%)")
print(f"   Test:          {test_size} ({CONFIG['test_ratio']*100:.0f}%)")

import fiftyone.utils.random as four

four.random_split(
    dataset,
    {
        "train": CONFIG['train_ratio'],
        "val": CONFIG['val_ratio'],
        "test": CONFIG['test_ratio']
    },
    seed=CONFIG['seed']
)

train_view = dataset.match_tags("train")
val_view = dataset.match_tags("val")
test_view = dataset.match_tags("test")

print(f"\nDivisione completata!")
print(f"   Campioni addestramento: {len(train_view)}")
print(f"   Campioni validazione:   {len(val_view)}")
print(f"   Campioni test:          {len(test_view)}")

print(f"\nVERIFICA VINCOLI SET TEST:")
print(f"   Minimo richiesto: {CONFIG['min_test_images']} immagini")
print(f"   Ottenuto: {len(test_view)} immagini")

if len(test_view) >= CONFIG['min_test_images']:
    print("   Vincolo soddisfatto!")
else:
    print(f"   ATTENZIONE: Set test troppo piccolo!")

test_class_counts = {}
for sample in test_view:
    if sample.detections:
        for det in sample.detections.detections:
            test_class_counts[det.label] = test_class_counts.get(det.label, 0) + 1

print(f"\nDistribuzione classi nel SET TEST:")
all_satisfied = True
for cls in CONFIG['classes']:
    count = test_class_counts.get(cls, 0)
    status = "OK" if count >= CONFIG['min_images_per_class'] else "ATTENZIONE"
    print(f"   [{status}] {cls:12s}: {count:3d} annotazioni")
    if count < CONFIG['min_images_per_class']:
        all_satisfied = False

print("="*60)

# Export in formato YOLO
print("\nEXPORT IN FORMATO YOLO\n")
print("="*60)

export_dir = CONFIG['base_path'] / CONFIG['dataset_name']
export_dir.mkdir(parents=True, exist_ok=True)

print(f"Directory export: {export_dir}\n")

has_segmentation = False
for sample in train_view.head(10):
    if sample.segmentations and len(sample.segmentations.detections) > 0:
        has_segmentation = True
        break

if has_segmentation:
    print("Dataset contiene maschere segmentazione - Export completo\n")
    label_field = "segmentations"
else:
    print("Dataset NON contiene maschere segmentazione - Solo detection\n")
    print("Nota: Addestramento segmentazione non sarà possibile\n")
    label_field = "detections"

for split_name, split_view in [("train", train_view), ("val", val_view), ("test", test_view)]:
    print(f"Esportazione {split_name}...")
    
    split_view.export(
        export_dir=str(export_dir),
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split_name,
        classes=CONFIG['classes']
    )
    
    print(f"   {split_name} esportato: {len(split_view)} immagini")

print(f"\nExport completato!")
print(f"Campo etichette utilizzato: {label_field}")
print("="*60)

# Creazione file data.yaml
yaml_path = export_dir / "data.yaml"

yaml_content = f"""path: {export_dir}
train: images/train
val: images/val
test: images/test

names:
  0: Horse
  1: Lizard
  2: Mouse
  3: Hamster
  4: Mule

nc: 5
"""

with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("File data.yaml creato:\n")
print(yaml_content)
print(f"Salvato in: {yaml_path}")

# Verifica struttura finale
print("\nSTRUTTURA DATASET FINALE:")
for root, dirs, files in os.walk(export_dir):
    level = root.replace(str(export_dir), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:3]:
        print(f'{subindent}{file}')
    if len(files) > 3:
        print(f'{subindent}... e altri {len(files)-3} file')